## Virus Gene Extraction, FASTA File Creation and Salmon Reference File Creation Workflow
This workflow extracts viral genes from various sources, saves them in FASTA format, and generates metadata for custom viruses. It also creates a "gentrome" file that includes human and viral transcriptomes for downstream RNA-seq quantification.

You need a main directory (DATA_DIR) with according sub directories. </br>
DATA_DIR/general_parts/ </br>
DATA_DIR/plots_with_decoy/ </br>
DATA_DIR/scripts/ </br>
DATA_DIR/human_genom/ </br>
DATA_DIR/viral_genes </br>
 </br>
These than also need certain files in them. </br>

General Data: </br>
    internal-24q4_v84-omicsdefaultmodelprofiles.csv (from taiga) </br>
    pr_table_subset.csv (some mapping file with the mapped ids (ProfileID, ModelID, SequenceID), from Gumbo) </br>
 </br>
Viral Genes: </br>
    viral_gene_identifiers.csv (get from GDC, https://gdc.cancer.gov/about-data/gdc-data-processing/gdc-reference-files, https://gdc.cancer.gov/system/files/public/file/GRCh83.d1.vd1_virus_decoy.txt) </br>
    DATA_DIR/viral_genes/{virus_name}/{virus_name}_raw.txt (any other virus genes that are not in the GDC list but still shall be included in the reference file later) </br>
    DATA_DIR/viral_genes/{virus_name}/pLenti-raw-genes.txt (any Lenti virus genes that shall be included in the reference file later) </br>

Make sure you have installed all packages and run this command in your terminal: </br>
*pip install requests pandas biopython*

In [ ]:
import os
import glob
import shutil
import requests
import zipfile
import csv
import subprocess
import pandas as pd
import re
import gzip
from Bio import SeqIO
from pathlib import Path
import urllib.request
from pathlib import Path

DATA_DIR = "/Users/ptrollma/experiment_data/virus_genes/" # adjust this to your directory path
GENERAL_DATA = DATA_DIR + "general_parts/"
META_DATA_DIR = DATA_DIR + "viral_genes/"
PLOT_DIR = DATA_DIR + "plots_with_decoy/"
HUMAN_GENE_DIR = DATA_DIR + "human_genom/"
SCRIPTS_DIR = DATA_DIR + "scripts/"

In [ ]:
# Check if directories exist
directories = {
    "META_DATA_DIR": DATA_DIR + "viral_genes/",
    "PLOT_DIR": DATA_DIR + "plots_with_decoy/",
    "HUMAN_GENE_DIR": DATA_DIR + "human_genom/",
    "GENERAL_DATA": DATA_DIR + "general_parts/",
    "SCRIPTS_DIR": DATA_DIR + "scripts/"
}

# Check if DATA_DIR exists
if os.path.exists(DATA_DIR):
    # Iterate over each directory and create it if it doesn't exist
    for dir_name, dir_path in directories.items():
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            print(f"{dir_name} created: {dir_path}")
else:
    print(f"{DATA_DIR} does not exist.")

### Code to list all bam files in the google bucket
You need the name and permission of the google bucket were the bam files are stored

In [ ]:
bucket_name = "gs://cclebams/rnasq_hg38/"   # Define your bucket name
user = 'philipp-trollmann'                  # Add a user that has permission for this bucket

# Run gsutil command to list all BAM files and save to CSV
with open(f"{SCRIPTS_DIR}bam_file_names.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["hg38_rna_bam"])  # Optional header
    result = subprocess.run(
            f"gsutil -u {user} ls {bucket_name}*Aligned.sortedByCoord.out.bam", 
            shell=True, capture_output=True, text=True, check=True
    )
    for file_path in result.stdout.splitlines():
        writer.writerow([file_path])

### Code to write column with ModelID and bam file location (only get the default condition measurement)
Make a file that has all the model IDs (cell lines) and the according bam file locations in them

In [ ]:
bam_files_df = pd.read_csv(f"{SCRIPTS_DIR}bam_file_names.csv")
profile_id = pd.read_csv(f"{GENERAL_DATA}internal-24q4_v84-omicsdefaultmodelprofiles.csv")
id_map = pd.read_csv(f"{GENERAL_DATA}pr_table_subset.csv")

# Create a dictionary for mapping column names to IDs
name_to_id = dict(zip(id_map['MainSequencingID'], id_map['ModelID']))

# Use to only take the untreated profiles, and only take the main profile of each cell line
profile_id_rna = profile_id[profile_id['ProfileType'] == 'rna']
profile_id_rna_merge = pd.merge(profile_id_rna, id_map, on='ProfileID', how='left')

# Get sequencing id from bucket path and map to ModelID, remove NaNs and duplicates
modelid_bucket_df = pd.DataFrame(columns= ["entity:sample_id", "hg38_rna_bam", "sequencing_id"])
modelid_bucket_df["hg38_rna_bam"] = bam_files_df["hg38_rna_bam"]
modelid_bucket_df['sequencing_id'] = modelid_bucket_df['hg38_rna_bam'].str.extract(r'(CDS-\w{6})')
modelid_bucket_df["entity:sample_id"] = modelid_bucket_df['sequencing_id'].map(name_to_id)
modelid_bucket_df = modelid_bucket_df[modelid_bucket_df['sequencing_id'].isin(profile_id_rna_merge['MainSequencingID'])]

modelid_bucket_df_sorted= modelid_bucket_df.sort_values(by='entity:sample_id').iloc[:,:2]
modelid_bucket_df_sorted.to_csv(f"{SCRIPTS_DIR}cell_line_google_bucket_index.csv", index=False)

### Download human genome

In [ ]:
# URLs to the files
transcriptome_url = "ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_38/gencode.v38.transcripts.fa.gz"
genome_url = "ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_38/GRCh38.primary_assembly.genome.fa.gz"

# Set the destination directory for downloads
download_dir = HUMAN_GENE_DIR
download_dir.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist

# Paths to save the downloaded files
transcriptome_file = download_dir + "gencode.v38.transcripts.fa.gz"
genome_file = download_dir + "GRCh38.primary_assembly.genome.fa.gz"

# Download the transcriptome file
urllib.request.urlretrieve(transcriptome_url, transcriptome_file)
print(f"Downloaded: {transcriptome_file}")

# Download the genome file
urllib.request.urlretrieve(genome_url, genome_file)
print(f"Downloaded: {genome_file}")

### Code to download all wanted viral gene sequences

In [ ]:
# Load the viral gene identifiers from CSV
viral_identifiers_df = pd.read_csv(META_DATA_DIR + "viral_gene_identifiers.csv")

# List of specific viruses to download
virus_downloads = ['CMV ( HHV-5)', 'EBV (HHV-4)', 'HBV', 'HCV-1', 'HCV-2', 'HIV-1', 'HIV-2', 'HHV-8, KSHV', 
                   'HTLV-1', 'MCV', 'SV40', 'HPV16', 'HPV18', 'HPV26', 'HPV53', 'HPV1', 'HPV2', 'HPV4', 
                   'HPV5', 'HPV6', 'HPV7', 'HPV9', 'HPV10', 'HPV24', 'HPV32', 'HPV34', 'HPV41', 'HPV48', 
                   'HPV49', 'HPV50', 'HPV54', 'HPV60', 'HPV61', 'HPV63', 'HPV88', 'HPV90', 'HPV92', 
                   'HPV96', 'HPV98', 'HPV99', 'HPV100', 'HPV101', 'HPV103', 'HPV104', 'HPV105', 'HPV108', 
                   'HPV109', 'HPV112', 'HPV113', 'HPV114', 'HPV115', 'HPV116', 'HPV121', 'HPV126', 
                   'HPV127', 'HPV128', 'HPV129', 'HPV131', 'HPV132', 'HPV134', 'HPV135', 'HPV136', 
                   'HPV137', 'HPV140', 'HPV144', 'HPV148', 'HPV154', 'HPV166', 'HPV167', 'HPV178', 
                   'HPV179']

# Get a dictionary of identifiers from the CSV
lookup_dict = dict(zip(viral_identifiers_df['Abbreviation'], viral_identifiers_df['RefSeq']))

# Filter the dictionary to include only the viruses in the virus_downloads list
result_dict = {key: lookup_dict[key] for key in virus_downloads if key in lookup_dict}

# Create an empty list to store metadata
metadata_list = []

def extract_genomic_range_from_fna(file_path):
    """Extracts the genomic range from the first line of the gene.fna file."""
    with open(file_path, 'r') as fna_file:
        first_line = fna_file.readline().strip()  # Read the first line
        match = re.search(r':c?(\d+)-(\d+)', first_line)  # Regex to capture the range
        if match:
            start, end = match.groups()
            return f"{start}-{end}"
        else:
            return None  # Handle case where range is not found

def get_gene_ids_from_refseq(refseq_id, page_size=200):
    """Fetch gene IDs with pagination to manage large responses."""
    all_gene_info = []
    page_number = 1  # Track the current page

    while True:
        # URL for the API request with a hypothetical pagination system
        url = f"https://api.ncbi.nlm.nih.gov/datasets/v2alpha/gene/accession/{refseq_id}"
        params = {
            "page_size": page_size,  # Limit the number of results per request
            "page": page_number      # Request the current page
        }
        headers = {
            'Accept': 'application/json'
        }
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            
            # Extract gene information from the response
            gene_info = [
                (
                    report['gene']['gene_id'], 
                    report['gene']['symbol'],
                    f"{report['gene']['annotations'][0]['genomic_locations'][0]['genomic_range']['begin']}-"
                    f"{report['gene']['annotations'][0]['genomic_locations'][0]['genomic_range']['end']}"
                )
                for report in data['reports']
            ]
            
            # Add current page's results to the total list
            all_gene_info.extend(gene_info)

            # Check if there's a "next" page or if we're done
            if 'next_page' in data:
                page_number += 1  # Move to the next page
            else:
                break  # No more pages, exit the loop
        else:
            print(f"Error: {response.status_code}, {response.text}")
            break
    
    return all_gene_info

def download_gene_data(gene_id, dir, gene_name, virus_name, genomic_range):
    # Replace '/' in gene and virus names with '_'
    sanitized_gene_name = gene_name.replace('/', '_')
    sanitized_virus_name = virus_name.replace('/', '_')
   
    url = f"https://api.ncbi.nlm.nih.gov/datasets/v2alpha/gene/id/{gene_id}/download"    
    params = {
        "include_annotation_type": ["FASTA_GENE"],  # Request the gene FASTA file
        "filename": f"{sanitized_virus_name}_{sanitized_gene_name}_{gene_id}_datasets.zip"
    }
    headers = {
        'Accept': 'application/json'
    }
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        zip_path = os.path.join(dir, f"{sanitized_virus_name}_{sanitized_gene_name}_{gene_id}_datasets.zip")
        unzip_path = os.path.join(dir, f"{sanitized_virus_name}_{sanitized_gene_name}_{gene_id}_datasets")

        # Save the zip file
        with open(zip_path, "wb") as file:
            file.write(response.content)
        print(f"Successfully downloaded data for gene {gene_name} from {virus_name}")

        # Extract the zip file
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        print(f"Extracted {zip_path} to {unzip_path}")
        
        # Delete the zip file
        os.remove(zip_path)
        print(f"Deleted zip file: {zip_path}")

        # Find the .fna file and extract genomic range
        fna_file_path = unzip_path + "/ncbi_dataset/data/gene.fna"
        file_genomic_range = extract_genomic_range_from_fna(fna_file_path)
        genomic_range = file_genomic_range if file_genomic_range else genomic_range  # Use file range if available

        # Append metadata to the list
        metadata_list.append({
            "Virus_name": virus_name,
            "Virus_id": refseq_id,
            "Gene_id": gene_id,
            "Gene_name": gene_name,
            "Genomic_range": genomic_range
        })
    
    else:
        print(f"Failed to download data for gene {gene_name} from {virus_name}: {response.status_code}")

# Loop through virus IDs and download gene files
for virus_name, refseq_id in result_dict.items():    
    download_dir = os.path.join(META_DATA_DIR + virus_name)  # Directory to save files
    os.makedirs(download_dir, exist_ok=True)  # Create directory if it doesn't exist

    gene_info = get_gene_ids_from_refseq(refseq_id)

    for gene_id, gene_name, genomic_range in gene_info:
        download_gene_data(gene_id, download_dir, gene_name, virus_name, genomic_range)

# Convert metadata list to DataFrame and save as CSV
metadata_df = pd.DataFrame(metadata_list)
metadata_df.to_csv(META_DATA_DIR + "viral_gene_metadata.csv", index=False)
print(f"Created metadata file.")

### Code to make gene fasta files from NCBI viruses without official NCBI ID
Copy the full text, of the NCBI entry for the virus you want a fasta file for each gene, to a '<virus_name>_raw.txt' file and put it in a directoroy DATA_DIR/viral_genes/{virus_name}/{virus_name}_raw.txt

In [ ]:
def extract_genes_and_save_fasta(file_path, virus_name, output_dir):
        
    version = None  # Initialize the version variable
    
    with open(file_path, 'r') as file:
        all_lines = file.readlines()

        lines = iter(all_lines)  # Convert the list to an iterator

        # Look for the VERSION line in the file
        for line in lines:
            if line.startswith("VERSION"):
                version = line.split()[1].strip()
                break  # Exit after finding the version
        
        # Initialize sequence string
        sequence = ""
        
        # Flag to start reading sequence after ORIGIN
        is_origin = False
        for line in lines:
            if line.startswith("ORIGIN"):
                is_origin = True
                continue
            
            # Stop reading when // is encountered
            if line.startswith("//"):
                break

            if is_origin:
                # Remove line numbers and spaces, then concatenate the sequence
                sequence += ''.join([char.upper() for char in line if char.isalpha()])

        lines = iter(all_lines)  # Reset the iterator to the start of the file

        # Process each CDS region after finding the version
        for line in lines:
            if line.startswith("     CDS             "):
                # Check if it's a 'join' case or a single range
                if "join(" in line:
                    # Extract ranges within 'join()', e.g., '266..13468,13468..21555'
                    ranges = line.split("join(")[1].split(")")[0].split(",")
                    #print(ranges)
                    # Convert ranges to start and end pairs
                    positions = [(int(r.split("..")[0]), int(r.split("..")[1])) for r in ranges]
                elif "complement(join(" in line:
                    # Extract ranges within 'join()', e.g., '266..13468,13468..21555'
                    ranges = line.split("complement(join(")[1].split("))")[0].split(",")
                    # Convert ranges to start and end pairs
                    positions = [(int(r.split("..")[0]), int(r.split("..")[1])) for r in ranges]
                elif "complement(" in line:
                    # Extract ranges within 'join()', e.g., '266..13468,13468..21555'
                    ranges = line.split("complement(")[1].split(")")[0].split(",")
                    # Convert ranges to start and end pairs
                    positions = [(int(r.split("..")[0]), int(r.split("..")[1])) for r in ranges]
                else:
                    # Single range case
                    start = int(line.split()[1].split("..")[0])
                    end = int(line.split()[1].split("..")[1])
                    positions = [(start, end)]

                # Initialize fields for product and gene_id
                product = None
                gene_id = "0000000"

                # Loop to find /product, /protein_id, and /db_xref="GeneID"
                while "/gene=" not in line and "/product=" not in line:                    
                    line = next(lines).strip()
                if "/gene=" in line:
                    product = line.split('"')[1].replace("/", "-")
                elif "/product=" in line:
                    product = line.split('"')[1].replace("/", "-")           

                while "/db_xref=" not in line and "/protein_id=" not in line:
                    line = next(lines).strip()
                if "/db_xref=" in line and "GeneID:" in line:
                    gene_id = line.split("GeneID:")[1].strip('"')
                elif "/protein_id=" in line and gene_id == "0000000":
                    gene_id = line.split('"')[1]  # Use protein ID if GeneID is missing
                

                # Create the gene directory and gene.fna file if both positions and product are found
                if positions and product:
                    # Format the subdirectory name: {virus_name}_{gene_name}_{gene_id}_datasets
                    gene_dir = os.path.join(output_dir, f"{virus_name}_{product}_{gene_id}_datasets")
                    os.makedirs(gene_dir, exist_ok=True)

                    # Define the path for the gene.fna file within the subdirectory structure
                    gene_data_dir = os.path.join(gene_dir, "ncbi_dataset/data")
                    os.makedirs(gene_data_dir, exist_ok=True)

                    fna_path = os.path.join(gene_data_dir, "gene.fna")

                    # Extract and concatenate sequences for each position range
                    extracted_sequence = "".join([sequence[start - 1:end] for start, end in positions])
                    
                    position_str = ",".join([f"{start}-{end}" for start, end in positions])

                    # Write the sequence to the gene.fna file
                    with open(fna_path, 'w') as fna_file:
                        fna_file.write(f">{version}:{position_str} {product} [organism={virus_name}]\n")
                        fna_file.write(f"{extracted_sequence}\n")

virus_name = "BoDV-1"  # Specify the virus name
file_path = META_DATA_DIR + f"{virus_name}/{virus_name}_raw.txt"
output_dir = META_DATA_DIR + f"{virus_name}/"
extract_genes_and_save_fasta(file_path, virus_name, output_dir)

In [ ]:
# Find all directories that contain a *_raw.txt file
directories_with_raw = [os.path.dirname(f) for f in glob.glob(os.path.join(META_DATA_DIR, "**/*_raw.txt"), recursive=True)]

for dir_path in directories_with_raw:
    # Delete only subdirectories within the directory
    for item in os.listdir(dir_path):
        item_path = os.path.join(dir_path, item)
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)
    
    # Extract virus name based on directory name
    virus_name = os.path.basename(dir_path)
    file_path = os.path.join(dir_path, f"{virus_name}_raw.txt")
    output_dir = dir_path
    extract_genes_and_save_fasta(file_path, virus_name, output_dir)

### Code to make gene fasta files from virus sequences form other sources (example: Lentivirus from Addgene website)
Copy all fasta sequences you want in one file 'DATA_DIR/viral_genes/{virus_name}/pLenti-raw-genes.txt'

In [ ]:
def extract_genes_and_save_fasta(file_path, virus_name, output_dir):
    gene_name = None
    sequence_lines = []
    virus_id = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            
            # Detect start of a new gene sequence
            if line.startswith("id="):
                virus_buffer = line[4:].strip()  # Update virus_id for each new gene block
                if virus_id == None:
                    virus_id = virus_buffer
            elif line.startswith(">"):
                # If we have a previous gene loaded, save it to a file
                if gene_name:
                    save_gene_fasta(gene_name, sequence_lines, virus_name, output_dir, virus_id)
                    virus_id = virus_buffer

                # Start new gene name and reset sequence
                gene_name = line[1:].strip()  # Strip '>' and any whitespace around the gene name
                sequence_lines = []  # Reset sequence storage for the new gene
            
            # Collect sequence lines for the current gene
            else:
                sequence_lines.append(line)

        # Save the last gene in the file
        if gene_name:
            save_gene_fasta(gene_name, sequence_lines, virus_name, output_dir, virus_id)


def save_gene_fasta(gene_name, sequence_lines, virus_name, output_dir, virus_id):
    # Format the subdirectory name: {virus_name}_{gene_name}_datasets
    gene_dir = os.path.join(output_dir, f"{virus_name}_{gene_name}_0000000_datasets")
    os.makedirs(gene_dir, exist_ok=True)

    # Define the path for the gene.fna file within the subdirectory structure
    gene_data_dir = os.path.join(gene_dir, "ncbi_dataset/data")
    os.makedirs(gene_data_dir, exist_ok=True)
    
    fna_path = os.path.join(gene_data_dir, "gene.fna")
    
    # Combine all sequence lines into a single string without newlines in between
    full_sequence = ''.join(sequence_lines)
    
    # Write the sequence to the gene.fna file
    with open(fna_path, 'w') as fna_file:
        fna_file.write(f">{virus_id}:0-0 {gene_name} [organism={virus_name}]\n")
        fna_file.write(f"{full_sequence}\n")


# Example usage
virus_name = "pLenti" 
file_path = META_DATA_DIR + f"{virus_name}/pLenti-raw-genes.txt"
output_dir = META_DATA_DIR + f"{virus_name}/"
extract_genes_and_save_fasta(file_path, virus_name, output_dir)

### Code to make the metadata file with all the viruses
Is needed if viruses were added manually (with code above) and not downloaded with API code above

In [ ]:
# Set the base directory where the viral gene directories are stored
base_dir = DATA_DIR + "viral_genes"

# Regular expression to match the directory name pattern
# This regex will capture:
# - virus_name as the part before the first underscore
# - gene_name as the part after the first underscore up to the gene_id
# - gene_id as the part before "_datasets" which can include letters, numbers, dots, or parentheses
pattern = r"^([^_]+)_(.+)_(\w+(\.\w+)?\(?\d*\)?)_datasets$"

# Initialize an empty list to store our data
data = []

# Walk through each virus directory in the base directory
for virus_dir in os.listdir(base_dir):
    virus_path = os.path.join(base_dir, virus_dir)
    
    # Ensure we're working with directories
    if os.path.isdir(virus_path):
        
        # Now, look for each gene subdirectory
        for gene_dir in os.listdir(virus_path):
            gene_path = os.path.join(virus_path, gene_dir)
            
            # Only proceed if it's indeed a directory
            if os.path.isdir(gene_path):
                
                # Match the directory name to extract virus_name, gene_name, and gene_id
                match = re.match(pattern, gene_dir)
                if not match:
                    print(f"Skipping directory {gene_dir} as it does not match the expected pattern.")
                    continue
                
                # Extract the parts from the regex match
                virus_name, gene_name, gene_id = match.groups()[:3]

                # Path to the gene.fna file
                fna_file_path = os.path.join(gene_path, "ncbi_dataset", "data", "gene.fna")
                
                # Check if the gene.fna file exists
                if os.path.isfile(fna_file_path):
                    # Read the first line of the gene.fna file
                    with open(fna_file_path, 'r') as fna_file:
                        first_line = fna_file.readline().strip()
                        
                        # Extract virus_id and genomic_range
                        try:
                            header_parts = first_line.split(" ")
                            virus_id, genomic_range = header_parts[0][1:].split(":")
                        except ValueError:
                            print(f"Skipping file {fna_file_path} due to unexpected header format.")
                            continue

                        # Append the extracted data to our list
                        data.append({
                            "Virus_name": virus_name,
                            "Virus_id": virus_id,
                            "Gene_id": gene_id,
                            "Gene_name": gene_name,
                            "Genomic_range": genomic_range
                        })

# Create a DataFrame from the data and save it to a CSV file
df = pd.DataFrame(data)
df.to_csv(f"{base_dir}/viral_gene_metadata.csv", index=False)
print("Data has been written to viral_gene_metadata.csv")

### Code to create a reference gene fasta file out of the human genome and different viral genes
Needed if you want to run the Salmon quantification without decoy, otherwise not needed

In [ ]:
gz_fasta_file = Path(HUMAN_GENE_DIR + 'gencode.v38.transcripts.fa.gz') # Path to the compressed human genome .gz FASTA file
viral_dir = META_DATA_DIR # Set the directory path where the .fa and .fna files are located
output_file = DATA_DIR + "reference_genes.fasta" # output file

# Search for all .fa and .fna files in the directory
# fasta_files = [gz_fasta_file] + list(viral_dir.glob("HPV16/**/gene.fna")) + list(viral_dir.glob("HPV18/**/gene.fna"))
fasta_files = [gz_fasta_file] + list(viral_dir.glob("**/**/gene.fna"))

# Convert the Path objects to strings
fasta_files = [str(f) for f in fasta_files]
print(f"Found {len(fasta_files)} files: {fasta_files}")

# Open the output file in write mode
with open(output_file, "w") as outfile:
    for fasta_file in fasta_files:
        if fasta_file.endswith(".gz"):
            # Handle compressed files
            with gzip.open(fasta_file, "rt") as handle:
                for record in SeqIO.parse(handle, "fasta"):
                    SeqIO.write(record, outfile, "fasta")
        else:
            # Handle uncompressed files
            with open(fasta_file, "r") as handle:
                for record in SeqIO.parse(handle, "fasta"):
                    SeqIO.write(record, outfile, "fasta")

print(f"Reference FASTA file created: {output_file}")

### Code to create a gentrome file (human + all virus)
Needed if you want to run the Salmon quantification with decoy

In [ ]:
# using this terminal line is way faster, though
# gunzip -c human_genom/gencode.v38.transcripts.fa.gz > gentrome.fa # Unzip the transcripts file and create gentrome.fa
# cat viral_genes/**/**/ncbi_dataset/data/gene.fna >> gentrome.fa # Concatenate viral gene files into gentrome.fa
# gunzip -c human_genom/GRCh38.primary_assembly.genome.fa.gz >> gentrome.fa # Unzip the genome file and append it to gentrome.fa
# # gzip gentrome.fa # Compress the final output file

# Paths to files
transcriptome_file = Path(HUMAN_GENE_DIR + 'gencode.v38.transcripts.fa.gz')
genome_file = Path(HUMAN_GENE_DIR + 'GRCh38.primary_assembly.genome.fa.gz')
viral_dir = META_DATA_DIR  # Directory containing viral gene .fa files
output_file = Path(DATA_DIR + 'gentrome.fa.gz')

# Open the output file in gzipped mode
with gzip.open(output_file, "wt") as outfile:
    # Write human transcriptome
    with gzip.open(transcriptome_file, "rt") as t_file:
        for record in SeqIO.parse(t_file, "fasta"):
            SeqIO.write(record, outfile, "fasta")    
    
    # Write viral gene files
    for viral_file in viral_dir.glob("**/**/gene.fna"):
        with open(viral_file, "r") as v_file:
            for record in SeqIO.parse(v_file, "fasta"):
                SeqIO.write(record, outfile, "fasta")
    
    # Write human genome
    with gzip.open(genome_file, "rt") as g_file:
        for record in SeqIO.parse(g_file, "fasta"):
            SeqIO.write(record, outfile, "fasta")

print(f"gentrome.fa.gz created at: {output_file}")

### Code to create a decoy list out of human genes
Needed if you want to run the Salmon quantification with decoy

In [ ]:
# Faster to use this terminal lines
# grep "^>" <(gunzip -c GRCh38.primary_assembly.genome.fa.gz) | cut -d " " -f 1 > decoys.txt
# sed -i.bak -e 's/>//g' decoys.txt

# Create decoy list
human_genome_file = Path(HUMAN_GENE_DIR + 'GRCh38.primary_assembly.genome.fa.gz')
decoy_output = DATA_DIR + "decoys.txt"

# Open output decoy file
with open(decoy_output, "w") as decoy_file:
    # Write human genome decoys
    for line in gzip.open(human_genome_file, 'rt'):
        if line.startswith(">"):
            decoy_file.write(line[1:].split(" ")[0] + "\n")

print(f"Decoy list created: {decoy_output}")

# To create the index with the decoy, use this Salmon command:
# salmon index -t gentrome.fa.gz -d decoys.txt -i salmon_index --gencode -k 31

### Code to check which cell lines were not aligned yet

In [ ]:
# Define the bucket path and output CSV file
bucket_path = "gs://virus_expression_results/transcript_quant_results/"
output_file = SCRIPTS_DIR + "missing_cell_lines.csv"

# Run a faster gsutil command without '-d' and process results to get directories only
command = ["gsutil", "ls", f"{bucket_path}"]
result = subprocess.run(command, capture_output=True, text=True)

# Extract directories by filtering out non-directory paths
directories = [line for line in result.stdout.splitlines() if line.endswith('/')]
directory_names = [line.replace(bucket_path, "").strip("/") for line in directories]

# Load the CSV file with all cell lines
all_cell_lines = pd.read_csv(f"{SCRIPTS_DIR}cell_line_google_bucket_index.csv")

# Filter rows where 'ModelID' is not in directory_names
missing_cell_lines = all_cell_lines[~all_cell_lines['entity:sample_id'].isin(directory_names)]

# Check if there are any missing cell lines and save or print message
if missing_cell_lines.empty:
    print("All cell lines were already aligned")
else:
    missing_cell_lines.to_csv(output_file, index=False)
    print(f"Missing cell lines saved to {output_file}")

### Code to map the ModelId to the Sequencing Id
If you need to map certain ModelIDs (ACH-00xxxx) to Sequencing IDs (CDS-xxxxx)

In [ ]:
id_map = pd.read_csv(f"{GENERAL_DATA}GumboQueryExport.csv")
cell_lines_df = pd.read_csv(f"{PLOT_DIR}/priority_cell_lines.csv")

# Create a dictionary for mapping column names to IDs
name_to_id = dict(zip(id_map['model_id'], id_map['sequencing_id']))
cell_lines_df['CDS_ID'] = cell_lines_df['ModelID'].map(name_to_id)
cell_lines_df.to_csv(f"{PLOT_DIR}/priority_cell_lines_mapped.csv")

### Code to write column with ModelID and bam file location and filter out already calculated cell lines

In [ ]:
bam_files_df = pd.read_csv(f"{SCRIPTS_DIR}bam_file_names.csv")
id_map = pd.read_csv(f"{GENERAL_DATA}GumboQueryExport.csv")
all_viral_genes_heatmap_df = pd.read_csv(f'{PLOT_DIR}all_viral_genes_heatmap_data.csv', index_col=0)

# Create a dictionary for mapping column names to IDs
name_to_id = dict(zip(id_map['sequencing_id'], id_map['model_id']))

# Get sequencing id from bucket path and map to ModelID, remove NaNs and duplicates
modelid_bucket_df = pd.DataFrame(columns= ["entity:sample_id", "hg38_rna_bam", "sequencing_id"])
modelid_bucket_df["hg38_rna_bam"] = bam_files_df["hg38_rna_bam"]
modelid_bucket_df['sequencing_id'] = modelid_bucket_df['hg38_rna_bam'].str.extract(r'(CDS-\w{6})')
modelid_bucket_df["entity:sample_id"] = modelid_bucket_df['sequencing_id'].map(name_to_id)
modelid_bucket_df = modelid_bucket_df.dropna(subset=['entity:sample_id']).drop_duplicates(subset='entity:sample_id')

# Filter out already calculated cell lines
modelid_bucket_df_filtered = modelid_bucket_df[~modelid_bucket_df['entity:sample_id'].isin(all_viral_genes_heatmap_df.index)]
modelid_bucket_df_sorted= modelid_bucket_df_filtered.sort_values(by='entity:sample_id').iloc[:,:2]
modelid_bucket_df_sorted.to_csv(f"{SCRIPTS_DIR}cell_line_google_bucket_index2.csv", index=False)

### Code to prepare the hpv and cellosaurus data and map the given IDs to the DepmapIDs

In [ ]:
hpv_cell_data = pd.read_csv(f"{DATA_DIR}HPV.csv")
id_map = pd.read_csv(f"{DATA_DIR}GumboQueryExport.csv")
cellosaurus = pd.read_csv(f"{DATA_DIR}Cellosaurus HPV cell lines.xlsx - Sheet1.csv")
id_map_ = pd.read_csv(f"{DATA_DIR}internal-24q2_v87-model.csv")

# Create a dictionary for mapping column names to IDs
name_to_id = dict(zip(id_map['sequencing_id'], id_map['model_id']))

# Rename columns of df1 using the mapping dictionary
hpv_cell_data = hpv_cell_data.rename(columns=name_to_id)
hpv_cell_data.to_csv(f"{DATA_DIR}HPV_mapped.csv")

# Create a dictionary for mapping column names to IDs
name_to_id_ = dict(zip(id_map_['RRID'], id_map_['ModelID']))

cellosaurus.columns = ['RRID', 'ModelID', '', '','Cancer','HPV','Type','','']
cellosaurus['ModelID'] = cellosaurus['RRID'].map(name_to_id_)
cellosaurus = cellosaurus.sort_values(by='ModelID')
cellosaurus.to_csv(f"{DATA_DIR}cellosaurus_mapped.csv")

hpv_cell_data_transposed = hpv_cell_data.T
hpv_cell_data_transposed.columns = hpv_cell_data_transposed.iloc[0] # Set the first row (which contains target IDs) as the new column names
hpv_cell_data_transposed = hpv_cell_data_transposed[1:] # Drop the first row which is now the header
hpv_cell_data_transposed.index.name = 'cell line' # Rename the index to something meaningful
hpv_cell_data_transposed = hpv_cell_data_transposed.sort_values(by='cell line')
hpv_cell_data_transposed.to_csv(f"{DATA_DIR}HPV_mapped_transposed.csv")


# Filter cell lines that have HPV genes in them
hpv_cell_data_filtered = hpv_cell_data.drop(columns=hpv_cell_data.columns[(hpv_cell_data == 0).all()])
hpv_cell_data_filtered.to_csv(f"{DATA_DIR}HPV_mapped_filtered.csv")

hpv_cell_data_filtered_transpose = hpv_cell_data_filtered.T
hpv_cell_data_filtered_transpose.columns = hpv_cell_data_filtered_transpose.iloc[0] # Set the first row (which contains target IDs) as the new column names
hpv_cell_data_filtered_transpose = hpv_cell_data_filtered_transpose[1:] # Drop the first row which is now the header
hpv_cell_data_filtered_transpose.index.name = 'cell line' # Rename the index to something meaningful
hpv_cell_data_filtered_transpose = hpv_cell_data_filtered_transpose.sort_values(by='cell line')
hpv_cell_data_filtered_transpose.to_csv(f"{DATA_DIR}HPV_mapped_filtered_transposed.csv")
